## Problem

Input : 
- เส้นทาง (ต้นทาง-ปลายทาง) : เปลี่ยนได้เพิ่มได้
- จำนวนงานของแต่ละเส้นทาง
- จุดเริ่มต้น

Output : 
- หาทุกทางที่ทำให้งานจบ
- ทุกทางยาวเท่าไหร่
- เอาทางที่ระยะทางซ้ำกันออก
- บอกด้วยว่าจากระยะทางทั้งหมด แบ่งเป็นระยะทางมีของและไม่มีของอย่างละเท่าไหร่

### Data 

In [7]:
import numpy as np
import pandas as pd

distances = np.load('data/distances.npy')
node_names = np.load('data/node_name.npy')
tasks = pd.read_csv('data/tasks.csv')

In [8]:
tasks

,counts,start,end
0,2,A,B
1,2,A,C
2,1,A,E
3,1,C,A
4,1,C,B


In [5]:
distances

array([[  0,  65,   7,  46, 284],
       [ 65,   0,  88,  67,  86],
       [  7,  88,   0, 121, 294],
       [ 46,  67, 121,   0, 194],
       [284,  86, 294, 194,   0]])

In [9]:
node_names

array(['A', 'B', 'C', 'D', 'E'], dtype='<U1')